# 2D flying-snake at Re=2000 and AoA=35deg (gmsh + OpenFOAM-2.0.1)

---

This notebook heavily relies on the Python package `snake`, available on [GitHub](https://github.com/mesnardo/snake).
`snake` was written to post-process and compare the numerical solution from 4 different softwares ([`cuIBM`](https://github.com/barbagroup/cuIBM), [`PetIBM`](https://github.com/barbagroup/PetIBM), [`IBAMR`](https://github.com/IBAMR/IBAMR), and [`OpenFOAM`](http://www.openfoam.com/)).

In [1]:
import os
import sys
import collections

import ipywidgets

import snake
from snake import miscellaneous
from snake.openfoam.simulation import OpenFOAMSimulation
from snake.cuibm.simulation import CuIBMSimulation

In [2]:
print('Python version: ' + sys.version)
print('snake version:' + snake.__version__)

Python version: 2.7.11 |Anaconda 2.5.0 (64-bit)| (default, Dec  6 2015, 18:08:32) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
snake version:0.1


---

### Description

We aim to compute the flow around a snake cross-section at Reynolds number 2000 and angle-of-attack 35deg using IcoFOAM, the laminar incompressible solver of OpenFOAM.

The mesh was generated with the free software GMSH and contains about 700k triangular cells.

---

### General information

* software: `OpenFOAM-2.0.1`
* post-processing software: `ParaView-4.1` (included in `OpenFOAM-2.3.1`)
* submission date: 2012/12/26 (mesnardo)
* running date: 2012/12/26
* machine: `yeager` (6 processes)

In [ ]:
simulation = OpenFOAMSimulation(description='IcoFOAM (GMSH)')

---

### Computational domain and mesh

* `GMSH`, triangular mesh, 700k cells
* domain: `[-15,15]x[-15,15]`
* cell characteristic-length near bluff-body: `0.004`.
* cell characteristic-length at external boundaries: `0.1`.

The mesh was generated with the GUI of GMSH.
We set the characteristic-length of the triangle edges near the bluff-body to be the minimum cell-width used in Krishnan et al. (2014).
The characteristic-length of the triangle edges at the external boundaries is set to 0.1.

To run with IcoFOAM, the 2D mesh is extruded in the thrid-direction with length 1.0.
We set only 1 cell in the third-direction and the front and back faces have an `empty` boundary condition.

The GMSH mesh in contained in the file `snakeAoA35_mesh700k.msh`.

To convert the mesh into an OpenFOAM format, run the utility (from the simulation directory):
    > gmshToFoam

---

### Boundary conditions

In [ ]:
boundary_info = '0/U'
%cat $boundary_info
boundary_info = '0/p'
%cat $boundary_info

---

### Simulation parameters

In [ ]:
simulation_info = 'system/controlDict'
%cat $simulation_info
simulation_info = 'system/fvSolution'
%cat $simulation_info
simulation_info = 'system/fvScheme'
%cat $simulation_info

---

### Running the case

To run the the case in parallel , we need to decompose the domain:
    > decomposeParDict

To run the case the simulation:
    > mpirun -n <nprocs> icoFOAM -parallel > log.icoFoam

To reconstruct the parallel solution:
    > reconstructPar

To generate the vorticity field at each time saved:
    > vorticity

---

### Post-processing

The simulation crashed after about 62.3 non-dimensional time-units because of an over-pressure at the outlet boundary condition (unphysical phenomenon that occurs every time a vortex leaves the computation domain).3

The numerical solution requires about 4.7G of storage capacity.

#### Forces

We plot the instantaneous force coefficients up to 80 time-units and compute the time-averaged values between 32 and 60 time-units.

In [ ]:
simulation.read_forces(display_coefficients=True,
                       force_coefficients_folder=os.path.join(simulation.directory, 
                                                              'forces'),
                       usecols=(0, 1, 2))
simulation.get_mean_forces(limits=[32.0, 60.0])
simulation.get_strouhal(limits=[32.0, 60.0], order=200)
simulation.plot_forces(display_coefficients=True,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       style='snakeReproducibility',
                       save_name='forceCoefficients')
dataframe = simulation.create_dataframe_forces(display_strouhal=True,
                                               display_coefficients=True)

We compare the IcoFOAM results with those reported in Krishnan et al. (2014).

In [ ]:
krishnan = CuIBMSimulation(description='Krishnan et al. (2014)')
krishnan.read_forces(file_path=os.path.join(os.environ['SNAKE'],
                                            'resources',
                                            'flyingSnake2d_cuibm_anush',
                                            'flyingSnake2dRe2000AoA35',
                                            'forces'))
krishnan.get_mean_forces(limits=[32.0, 60.0])
krishnan.get_strouhal(limits=[32.0, 60.0], order=200)
simulation.plot_forces(display_coefficients=True,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       other_simulations=krishnan,
                       other_coefficients=2.0,
                       style='snakeReproducibility',
                       save_directory=os.path.join(simulation.directory,
                                                   'images'),
                       save_name='forceCoefficientsCompareKrishnanEtAl2014')
dataframe2 = krishnan.create_dataframe_forces(display_strouhal=True,
                                              display_coefficients=True,
                                              coefficient=2.0)

In [ ]:
dataframes = dataframe.append(dataframe2)
print(dataframes)

In [ ]:
images_directory = os.path.join(simulation.directory,
                                'images')
figures = collections.OrderedDict()
figures['present'] = os.path.join(images_directory,
                                  'forceCoefficients.png')
figures['present + Krishnan et al. (2014)'] = os.path.join(images_directory,
                                                           'forceCoefficientsCompareKrishnanEtAl2014.png')
ipywidgets.interact(miscellaneous.display_image, figure=figures);

#### Vorticity field

The cells below plot the vorticity field in the region [-2,15]x[-5,5] at every saved time-unit with ParaFOAM in batch mode and display the .png files in an interactive way.

In [ ]:
simulation.plot_field_contours_paraview('vorticity',
                                        field_range=(-5.0, 5.0),
                                        view=(-2.0, -5.0, 15.0, 5.0),
                                        width=600,
                                        colormap='RdBu_r')

In [ ]:
miscellaneous.displayer(os.path.join(simulation.directory,
                                     'images',
                                     'vorticity_-2.00_-5.00_15.00_5.00'))

#### Pressure field

The cells below plot the pressure field in the region [-2,15]x[-5,5] and in the entire domain at every saved time-unit with ParaFOAM in batch mode and display the .png files in an interactive way.

In [ ]:
simulation.plot_field_contours_paraview('pressure',
                                        field_range=(-1.0, 0.5),
                                        view=(-2.0, -5.0, 15.0, 5.0),
                                        width=600,
                                        colormap='viridis')

In [ ]:
miscellaneous.displayer(os.path.join(simulation.directory,
                                     'images',
                                     'pressure_-2.00_-5.00_15.00_5.00'))

In [ ]:
simulation.plot_field_contours_paraview('pressure',
                                        field_range=(-1.0, 0.5),
                                        view=(-15.0, -15.0, 15.0, 15.0),
                                        width=600,
                                        colormap='viridis')

In [ ]:
miscellaneous.displayer(os.path.join(simulation.directory,
                                     'images',
                                     'pressure_-15.00_-15.00_15.00_15.00'))